In this basic functionalities of [PyTorch](https://pytorch.org) will be explored for small datasets. The goal is to work on how to load images, pre-process the images and perform data augmentation. Finally, a deep neural network will be trained to perform image classification.




## Data loading

In this task, the data will be loaded in two different way. At first, the `torchvision.datasets` subclasses will be used to load a dataset. Second, we will write are own dataset loader that performs the same activity.


### a) Built-in Torch Dataloader

In [1]:
import torch
import torchvision
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms, utils, datasets
import numpy as np
from PIL import Image
import matplotlib.pyplot as plt
import torchvision.transforms.functional as F

In [ ]:

FashionMNIST_dataset = torchvision.datasets.FashionMNIST(root="./data", train=True, transform=transforms.ToTensor(), download=True)

MNIST_dataset = torchvision.datasets.MNIST(root="./data", train=True, transform=transforms.ToTensor(), download=True)

# Dataloader MNIST
MNIST_dataloader = DataLoader(MNIST_dataset, batch_size=32, shuffle=True)


# Dataloader Fashion-MNIST
FashionMNIST_dataloader = DataLoader(FashionMNIST_dataset, batch_size=32, shuffle=True)




In [ ]:
# Visualize MNIST images

for images, labels in MNIST_dataloader:
    figs, axis = plt.subplots(1,3, figsize=(9,9))
    for i, image in enumerate(images[:3]):
        axis[i].imshow(image.numpy().transpose(1,2,0))
        axis[i].set_title(MNIST_dataset.classes[labels[i]])
        axis[i].axis('off')
    plt.show()
    break

In [ ]:
# Visualize Fashion MNIST images
for images, labels in FashionMNIST_dataloader:
    figures, axss = plt.subplots(1,3, figsize = (9,9))
    for i, image in enumerate(images[:3]):
        axss[i].imshow(image.numpy().transpose(1,2,0))
        axss[i].set_title(FashionMNIST_dataset.classes[labels[i]])
        axss[i].axis("off")
    plt.show()
    break

### b) Customized Dataset

In [ ]:
import torch
from torch.utils.data import Dataset
import os
import urllib.request
import gzip

In [2]:
class CustomClassification(Dataset):
    def __init__(self, root_dir, train=True, transform=None, url_input=None):
        self.root_dir = root_dir
        self.train = train
        self.transform = transform
        self.url_input = url_input

        if self.train:
            self.data_file = 'train-images-idx3-ubyte.gz'
            self.label_file = 'train-labels-idx1-ubyte.gz'
        else:
            self.data_file = 't10k-images-idx3-ubyte.gz'
            self.label_file = 't10k-labels-idx1-ubyte.gz'

        self.data_path = os.path.join(self.root_dir, self.data_file)
        self.label_path = os.path.join(self.root_dir, self.label_file)

        if not os.path.exists(self.data_path) or not os.path.exists(self.label_path):
            print("Downloading dataset...")
            os.makedirs(self.root_dir, exist_ok=True)
            self.download_dataset()

        with gzip.open(self.label_path, 'rb') as f:
            magic_num, num_labels = np.frombuffer(f.read(8), dtype=np.int32)
            self.labels = np.frombuffer(f.read(), dtype=np.uint8)


        if self.train:
            with gzip.open(self.data_path, 'rb') as f:
                magic_num, num_images, num_rows, num_cols = np.frombuffer(f.read(16), dtype=np.int32)
                self.data = np.frombuffer(f.read(), dtype=np.uint8).reshape(60000, 28, 28)

        if self.train==False :
            with gzip.open(self.data_path, 'rb') as f:
                magic_num, num_images, num_rows, num_cols = np.frombuffer(f.read(16), dtype=np.int32)
                self.data = np.frombuffer(f.read(), dtype=np.uint8).reshape(10000, 28, 28)


    def __len__(self):
        return len(self.labels)

    def download_dataset(self):
        url = self.url_input
        files = [self.data_file, self.label_file]

        for file in files:
            file_url = url + file
            file_path = os.path.join(self.root_dir, file)

            urllib.request.urlretrieve(file_url, file_path)

    def __getitem__(self,idx):
        image = self.data[idx]
        label = self.labels[idx]

        if self.transform:
            image = self.transform(image)

        return image, label




# Load MNIST using custom dataset loader

mnist_dataset_custom = CustomClassification(root_dir='./data/mnist', train=True, transform=transforms.ToTensor(), url_input= 'http://yann.lecun.com/exdb/mnist/')
mnist_dataloader_custom = DataLoader(mnist_dataset_custom, batch_size=32, shuffle=True)

# Visualize CUSTOM MNIST images

for images, labels in mnist_dataloader_custom:
    figures, axss = plt.subplots(1,8, figsize = (9,9))
    for i, image in enumerate(images[:8]):
        axss[i].imshow(image.numpy().transpose(1,2,0))
        axss[i].set_title(MNIST_dataset.classes[labels[i]])
        axss[i].axis("off")
    plt.show()
    break


In [ ]:
# Load CUSTOM Fashion MNIST using custom dataset loader

Fmnist_dataset_custom = CustomClassification(root_dir='./data/FashionMNIST', train=True, transform=transforms.ToTensor(), url_input='http://fashion-mnist.s3-website.eu-central-1.amazonaws.com/')
Fmnist_dataloader_custom = DataLoader(Fmnist_dataset_custom, batch_size=32, shuffle=True)

# Visualize CUSTOM MNIST images

for images, labels in Fmnist_dataloader_custom:
    figures, axss = plt.subplots(1,8, figsize = (9,9))
    for i, image in enumerate(images[:8]):
        axss[i].imshow(image.numpy().transpose(1,2,0))
        axss[i].set_title(FashionMNIST_dataset.classes[labels[i]])
        axss[i].axis("off")
    plt.show()
    break

## 2. Model training

In this task, the [Fashion-MNIST](https://github.com/zalandoresearch/fashion-mnist) dataset will be employed for learning a classifier. The classifier will be a convolutional neural network.



In [ ]:
# Import libraries
import torch
import torch.nn as nn
import torch.optim as optim
import matplotlib.pyplot as plt

# Define the CNN model
class CNN_model(nn.Module):
    def __init__(self):
        super(CNN_model, self).__init__()
        self.conv1 = nn.Conv2d(1, 32, 3, padding=1)
        self.conv2 = nn.Conv2d(32, 64, 3, padding=1)
        self.pool = nn.MaxPool2d(2, 2)
        self.fc1 = nn.Linear(64 * 7 * 7, 128)
        self.fc2 = nn.Linear(128, 10)

    def forward(self, x):
        x = self.pool(nn.functional.relu(self.conv1(x)))
        x = self.pool(nn.functional.relu(self.conv2(x)))
        x = x.view(-1, 64 * 7 * 7)
        x = nn.functional.relu(self.fc1(x))
        x = self.fc2(x)
        return x

# Set device (CPU or GPU)
# device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Define funtion to Train the model,
# Here output of this function will be the loss and accuracy history of train and test data
def train_evaluate_model(model, train_dataloader, train_dataset, test_dataloader, test_dataset, num_epochs=10):
    criterion = nn.CrossEntropyLoss()
    optimizer = optim.Adam(model.parameters())
    train_losses = []
    test_losses = []
    train_accuracies = []
    test_accuracies = []
    for epoch in range(num_epochs):
        train_loss = 0
        train_correct = 0
        train_total = 0
        for images, labels in train_dataloader:
            images, labels = images.to(device), labels.to(device)
            optimizer.zero_grad()
            outputs = model(images)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()
            train_loss += loss.item()
            _, predicted = torch.max(outputs.data, 1)
            train_total += labels.size(0)
            train_correct += (predicted == labels).sum().item()
        train_accuracy = 100 * train_correct / train_total
        train_losses.append(train_loss / len(train_dataset))
        train_accuracies.append(train_accuracy)

        test_loss = 0
        test_correct = 0
        test_total = 0
        with torch.no_grad():
            for images, labels in test_dataloader:
                images, labels = images.to(device), labels.to(device)
                outputs = model(images)
                loss = criterion(outputs, labels)
                test_loss += loss.item()
                _, predicted = torch.max(outputs.data, 1)
                test_total += labels.size(0)
                test_correct += (predicted == labels).sum().item()
            test_accuracy = 100 * test_correct / test_total
            test_accuracies.append(test_accuracy)
            test_losses.append((test_loss) / (len(test_dataset)))
        print("\nEpoch:",epoch," completed. Train accuracy:",train_accuracies[epoch],", Train loss", train_losses[epoch])
        print("Test accuracy:", test_accuracies[epoch], ", Test loss:", test_losses[epoch])
    return train_accuracies, train_losses, test_accuracies, test_losses




In [ ]:
# Load FashionMNIST dataset (Train and Test) using CUSTOM dataset loader
fashion_mnist_dataset = CustomClassification(root_dir='./data/fashion-mnist', train=True, transform=transforms.ToTensor(), url_input='http://fashion-mnist.s3-website.eu-central-1.amazonaws.com/')
fashion_mnist_test_dataset = CustomClassification(root_dir='./data/fashion-mnist', train=False, transform=transforms.ToTensor(), url_input='http://fashion-mnist.s3-website.eu-central-1.amazonaws.com/')

fashion_mnist_dataloader = DataLoader(fashion_mnist_dataset, batch_size=32, shuffle=True)
fashion_mnist_test_dataloader = DataLoader(fashion_mnist_test_dataset, batch_size=32, shuffle=False)

# Define the CNN model
CNNmodel_FashionMNIST = CNN_model().to(device)

num_epochs=10
train_accuracies_FashionMNIST, train_losses_FashionMNIST, test_accuracies_FashionMNIST, test_losses_FashionMNIST = train_evaluate_model(CNNmodel_FashionMNIST,
                                                                                                                                        fashion_mnist_dataloader,
                                                                                                                                        fashion_mnist_dataset,
                                                                                                                                        fashion_mnist_test_dataloader,
                                                                                                                                        fashion_mnist_test_dataset, num_epochs)

In [ ]:
plt.plot(np.arange(0,num_epochs), train_accuracies_FashionMNIST, "r", label="Training Accuracy")
plt.plot(np.arange(0,num_epochs), test_accuracies_FashionMNIST, "b", label="Test Accuracy")
plt.title("Training and Test Accuracies - Fashion MNIST -without Data Augmentation")
plt.xlabel("Epochs")
plt.ylabel("Accuracy (%)")
plt.legend()
plt.show()

In [ ]:
plt.plot(np.arange(0,num_epochs), train_losses_FashionMNIST, "r",  label="Training Loss")
plt.plot(np.arange(0,num_epochs), test_losses_FashionMNIST, "b",  label="Test Loss")
plt.title("Training and Test Losses - Fashion MNIST -without Data Augmentation")
plt.xlabel("Epochs")
plt.ylabel("Loss/ Error")
plt.legend()
plt.show()

In [ ]:
print("The train accuracy after 10 epochs of FashionMNIST dataset without augmentation:", train_accuracies_FashionMNIST[9])
print("The train loss after 10 epochs of FashionMNIST dataset without augmentation:", train_losses_FashionMNIST[9])
print("The final test accuracy of FashionMNIST dataset without augmentation:", test_accuracies_FashionMNIST[9])
print("The final test loss of FashionMNIST dataset without augmentation:", test_losses_FashionMNIST[9])


## 3. Data Augmentation


The datasets [Fashion-MNIST](https://github.com/zalandoresearch/fashion-mnist) and [MNIST](http://yann.lecun.com/exdb/mnist/) will be the testbed to investigate the impact of data augmentation.



In [ ]:
print("The train accuracy after 10 epochs of FashionMNIST dataset without augmentation:", train_accuracies_FashionMNIST[9])
print("The train loss after 10 epochs of FashionMNIST dataset without augmentation:", train_losses_FashionMNIST[9])
print("The final test accuracy of FashionMNIST dataset without augmentation:", test_accuracies_FashionMNIST[9])
print("The final test loss of FashionMNIST dataset without augmentation:", test_losses_FashionMNIST[9])


In [ ]:
# Load MNIST dataset using CUSTOM dataset loader
mnist_dataset = CustomClassification(root_dir='./data/mnist', train=True, transform=transforms.ToTensor(), url_input='http://yann.lecun.com/exdb/mnist/')
mnist_test_dataset = CustomClassification(root_dir='./data/mnist', train=False, transform=transforms.ToTensor(), url_input='http://yann.lecun.com/exdb/mnist/')

mnist_dataloader = DataLoader(fashion_mnist_dataset, batch_size=32, shuffle=True)
mnist_test_dataloader = DataLoader(fashion_mnist_test_dataset, batch_size=32, shuffle=False)

# Define the CNN model
CNNmodel_MNIST = CNN_model().to(device)

num_epochs=10
train_accuracies_MNIST, train_losses_MNIST, test_accuracies_MNIST, test_losses_MNIST = train_evaluate_model(CNNmodel_MNIST, mnist_dataloader,
                                                                                                            mnist_dataset, mnist_test_dataloader,
                                                                                                            mnist_test_dataset, num_epochs)


In [ ]:
plt.plot(np.arange(0,num_epochs), train_accuracies_MNIST, "r", label="Training Accuracy")
plt.plot(np.arange(0,num_epochs), test_accuracies_MNIST, "b", label="Test Accuracy")
plt.title("Training and Test Accuracies - MNIST without Data augmentation")
plt.xlabel("Epochs")
plt.ylabel("Accuracy (%)")
plt.legend()
plt.show()

In [ ]:
plt.plot(np.arange(0,num_epochs), train_losses_MNIST, "r",  label="Training Loss")
plt.plot(np.arange(0,num_epochs), test_losses_MNIST, "b",  label="Test Loss")
plt.title("Training and Test Losses - MNIST without Data augmentation")
plt.xlabel("Epochs")
plt.ylabel("Loss/ Error")
plt.legend()
plt.show()

In [ ]:
import torch
import torchvision
from torchvision import transforms

# Define the data transforms for data augmentation
transform_train = transforms.Compose([
    transforms.RandomCrop(28, padding=5),
    transforms.RandomVerticalFlip(),
    transforms.ToTensor()
])

transform_test = transforms.Compose([
    transforms.ToTensor()
])

# Load the MNIST dataset with data augmentation
mnist_dataset_dataAug = torchvision.datasets.MNIST(root='./data', train=True, download=False, transform=transform_train)
mnist_dataset_test_dataAug = torchvision.datasets.MNIST(root='./data', train=False, download=False, transform=transform_test)

# Create data loaders
mnist_dataloader_dataAug = torch.utils.data.DataLoader(mnist_dataset_dataAug, batch_size=32, shuffle=True)
mnist_dataloader_test_dataAug = torch.utils.data.DataLoader(mnist_dataset_test_dataAug, batch_size=32, shuffle=False)

for images, labels in mnist_dataloader_dataAug:
    figures, axss = plt.subplots(1,8, figsize = (9,9))
    for i, image in enumerate(images[:8]):
        axss[i].imshow(image.numpy().transpose(1,2,0))
        axss[i].set_title(MNIST_dataset.classes[labels[i]])
        axss[i].axis("off")
    plt.show()
    break



In [ ]:
# Define the CNN model
CNNmodel_MNIST_data_aug = CNN_model().to(device)

num_epochs=10
train_accuracies_MNIST_dataAug, train_losses_MNIST_dataAug, test_accuracies_MNIST_dataAug, test_losses_MNIST_dataAug = train_evaluate_model(CNNmodel_MNIST_data_aug, mnist_dataloader_dataAug,
                                                                                                            mnist_dataset_dataAug, mnist_dataloader_test_dataAug,
                                                                                                            mnist_dataset_test_dataAug, num_epochs)


In [ ]:
plt.plot(np.arange(0,num_epochs), train_accuracies_MNIST_dataAug, "r", label="Training Accuracy")
plt.plot(np.arange(0,num_epochs), test_accuracies_MNIST_dataAug, "b", label="Test Accuracy")
plt.title("Training and Test Accuracies - MNIST with DATA augmentation")
plt.xlabel("Epochs")
plt.ylabel("Accuracy (%)")
plt.legend()
plt.show()

In [ ]:
plt.plot(np.arange(0,num_epochs), train_losses_MNIST_dataAug, "r",  label="Training Loss")
plt.plot(np.arange(0,num_epochs), test_losses_MNIST_dataAug, "b",  label="Test Loss")
plt.title("Training and Test Losses - MNIST  with DATA augmentation")
plt.xlabel("Epochs")
plt.ylabel("Loss/ Error")
plt.legend()
plt.show()

In [ ]:
# Load the FashionMNIST dataset with data augmentation
Fashion_dataset_dataAug = torchvision.datasets.FashionMNIST(root='./data', train=True, download=False, transform=transform_train)
Fashion_dataset_test_dataAug = torchvision.datasets.FashionMNIST(root='./data', train=False, download=False, transform=transform_test)

# Create data loaders
Fashion_dataloader_dataAug = torch.utils.data.DataLoader(Fashion_dataset_dataAug, batch_size=32, shuffle=True)
Fashion_dataloader_test_dataAug = torch.utils.data.DataLoader(Fashion_dataset_test_dataAug, batch_size=32, shuffle=False)

for images, labels in Fashion_dataloader_dataAug:
    print(images.shape)
    figures, axss = plt.subplots(1,8, figsize = (9,9))
    for i, image in enumerate(images[:8]):
        axss[i].imshow(image.numpy().transpose(1,2,0))
        axss[i].set_title(FashionMNIST_dataset.classes[labels[i]])
        axss[i].axis("off")
    plt.show()
    break

In [ ]:
# Define the CNN model
CNNmodel_FashionMNIST_data_aug = CNN_model().to(device)

num_epochs=10
train_accuracies_Fashion_dataAug, train_losses_Fashion_dataAug, test_accuracies_Fashion_dataAug, test_losses_Fashion_dataAug = train_evaluate_model(CNNmodel_FashionMNIST_data_aug, Fashion_dataloader_dataAug,
                                                                                                            Fashion_dataset_dataAug, Fashion_dataloader_test_dataAug,
                                                                                                            Fashion_dataset_test_dataAug, num_epochs)


In [ ]:
plt.plot(np.arange(0,num_epochs), train_accuracies_Fashion_dataAug, "r", label="Training Accuracy")
plt.plot(np.arange(0,num_epochs), test_accuracies_Fashion_dataAug, "b", label="Test Accuracy")
plt.title("Training and Test Accuracies - Fashion MNIST with DATA augmentation")
plt.xlabel("Epochs")
plt.ylabel("Accuracy (%)")
plt.legend()
plt.show()

In [ ]:
plt.plot(np.arange(0,num_epochs), train_losses_Fashion_dataAug, "r",  label="Training Loss")
plt.plot(np.arange(0,num_epochs), test_losses_Fashion_dataAug, "b",  label="Test Loss")
plt.title("Training and Test Losses - Fashion MNIST  with DATA augmentation")
plt.xlabel("Epochs")
plt.ylabel("Loss/ Error")
plt.legend()
plt.show()

In [ ]:
print("The train accuracy of MNIST dataset without augmentation:", train_accuracies_MNIST[9])
print("The test accuracy of MNIST dataset without augmentation:", test_accuracies_MNIST[9])

print("\nThe train accuracy of MNIST dataset WITH DATA augmentation:", train_accuracies_MNIST_dataAug[9])
print("The test accuracy of MNIST dataset WITH DATA augmentation:", test_accuracies_MNIST_dataAug[9])

print("\nThe train accuracy of FASHION MNIST dataset without augmentation:", train_accuracies_FashionMNIST[9])
print("The test accuracy of FASHION MNIST dataset without augmentation:", test_accuracies_FashionMNIST[9])

print("\nThe train accuracy of FASHION MNIST dataset WITH DATA augmentation:", train_accuracies_Fashion_dataAug[9])
print("The test accuracy of FASHION MNIST dataset WITH DATA augmentation:", test_accuracies_Fashion_dataAug[9])